In [106]:
# imports
from utils.pointcloud import BBox, grid_subsample_simple
from data.fwf_dataset import FwfDataset
from models.fgf import FGFeatNetwork
from omegaconf import OmegaConf
import tracemalloc

# tracemalloc.start()


import numpy as np
import json
import os


# build config
cfg = OmegaConf.load("./config/default.yaml")
cfg = OmegaConf.merge(cfg, OmegaConf.load(cfg.data.split))
with open(os.path.join(cfg.data.dataset_root, 'class_dict.json'),'r') as f:
    cfg = OmegaConf.merge(cfg, OmegaConf.create({'data':{'label_schema':json.load(f)}}))


print(OmegaConf.to_yaml(cfg))






general:
  exp_description: ''
  device: cuda
  batch_size: 1024
  weight_decay: 0.0005
  max_epochs: 50
  early_stopping_minDelta: 0.01
  num_workers: 4
data:
  dataset_root: ../../02_Datasets/FWF_Subsampled/0.01_simple/
  split: ./config/default_split.yaml
  preprocessing:
    _transformsTraining_:
      xyz:
        TransCenterXyz: {}
        TransZRotation: {}
        TransScaling: {}
        TransGaussianNoise:
          mean: 0
          std: 0.002
      rgb:
        TransGaussianNoise:
          mean: 0
          std: 0.02
        TransFeatureDropout:
          dropout_prob: 0.1
      wfm:
        TransSignalScaling: {}
      normals:
        TransGaussianNoise:
          mean: 0
          std: 0.05
      incAngles:
        TransGaussianNoise:
          mean: 0
          std: 0.05
        TransStandardize:
          mean: 2.07
          std: 0.38
      distanceFromScanner:
        TransGaussianNoise:
          mean: 12.13
          std: 12.2
    _transformsValidation_:
      xyz

In [107]:
train_ds = FwfDataset(cfg, cfg.data.preprocessing._transformsTraining_, cfg.data._trainProjects_)
val_ds = FwfDataset(cfg, cfg.data.preprocessing._transformsValidation_, cfg.data._valProjects_)


Loading '2024-03-22_FW_Koenigshuegel.FwfProj'; Bounding box IDs = default
Loading '2024-04-05_FW_Westbahnhof_02.FwfProj'; Bounding box IDs = default
Loading '2024-04-05_FW_Westbahnhof_03.FwfProj'; Bounding box IDs = default
Loading '2024-04-05_FW_Westbahnhof_04.FwfProj'; Bounding box IDs = default
Loading '2024-04-05_FW_Westbahnhof_05.FwfProj'; Bounding box IDs = default
Loading '2024-05-10_FW_RWTH_Zentrum_01.FwfProj'; Bounding box IDs = default
Loading '2024-07-31_FW_Bruecke_Koenigstr.FwfProj'; Bounding box IDs = [0, 2]
Loading '2024-08-02_FW_Bruecke_A44_VerlautenheidenerStr.FwfProj'; Bounding box IDs = default
Loading '2024-08-02_FW_Bruecke_Deltourserb.FwfProj'; Bounding box IDs = default
Loading '2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj'; Bounding box IDs = [1]
Loading '2024-08-02_FW_Bruecke_RotheErde.FwfProj'; Bounding box IDs = default
Loading '2024-08-02_FW_Bruecke_Rottstrasse.FwfProj'; Bounding box IDs = default
Loading '2023-08-28_FW_EingangBauing.FwfProj'; Bounding box IDs 

In [108]:
# normals, inc angles
train_ds.subsample_grid(cfg.data.query_grid_size, save_inv=True)
train_ds.compute_neibors_knn(k=cfg.data.num_neib_normalsComputation)
train_ds.compute_normals_knn()
train_ds.compute_incAngles()


val_ds.subsample_grid(cfg.data.query_grid_size, save_inv=True)
val_ds.compute_neibors_knn(k=cfg.data.num_neib_normalsComputation)
val_ds.compute_normals_knn()
val_ds.compute_incAngles()
val_ds.compute_neibors_knn(k=cfg.data.num_neib_featureExtraction, verbose=True)

Computing neibors for '2024-03-22_FW_Koenigshuegel.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-04-05_FW_Westbahnhof_02.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-04-05_FW_Westbahnhof_03.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-04-05_FW_Westbahnhof_04.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-04-05_FW_Westbahnhof_05.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-05-10_FW_RWTH_Zentrum_01.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-07-31_FW_Bruecke_Koenigstr.FwfProj::bboxId=000' @ k=20
Computing neibors for '2024-07-31_FW_Bruecke_Koenigstr.FwfProj::bboxId=002' @ k=20
Computing neibors for '2024-08-02_FW_Bruecke_A44_VerlautenheidenerStr.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-08-02_FW_Bruecke_Deltourserb.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj::bboxId=001' @ k=20
Computing neibors for '2024-08-02_FW_Bruecke_RotheErde.FwfProj::defaultBbox'

In [6]:
from torch.utils.data import DataLoader

# create model
model = FGFeatNetwork(cfg=cfg,
    num_input_feats = train_ds[0]['features_neibors'].shape[-1],
    ).to(device=cfg.general.device)


In [5]:
# train_dl = DataLoader(train_ds, batch_size=cfg.general.batch_size, num_workers=0) --> ca. 6it/sec, no mem problem
# train_dl = DataLoader(train_ds, batch_size=cfg.general.batch_size, num_workers=2) --> ca. 12 it/sec, doable but still knapp

In [9]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import torch
from tqdm import tqdm
from utils.metrics import get_multilevel_metrics, print_metrics, combine_metrics_list
from time import time



criterion = CrossEntropyLoss()

optim = Adam(params=model.parameters(), weight_decay=cfg.general.weight_decay)

val_dl = DataLoader(val_ds, batch_size=cfg.general.batch_size, num_workers=4)

for epoch in range(cfg.general.max_epochs):
    print(f"Epoch-{epoch:03}")

    start = time()
    print(f'Sampling training dataset.',end=' ')
    # subsample a random percentage of points
    train_ds.subsample_random(0.01)
    train_ds.compute_neibors_knn(k=cfg.data.num_neib_featureExtraction, verbose=False)
    train_dl = DataLoader(train_ds, batch_size=cfg.general.batch_size, num_workers=4, pin_memory=True)
    print(f'Done. Took {time()-start:.2f}.')
    # train
    epoch_train_loss = []
    epoch_metrics = []
    model.train()
    for i, batch in enumerate(tqdm(train_dl, desc=f"{'Training':<15}", leave=True)):
        optim.zero_grad()
        # put batch on device
        for k, v in batch.items():
            batch[k] = v.to(device=cfg.general.device)

        # forward pass
        out = model(batch)
        
        # agregate loss on all levels
        loss = torch.tensor(0.).to(device=cfg.general.device)
        for k in out.keys():
            output = out[k]
            gt = batch[k] # type:ignore
            loss += criterion(output, gt)
        
        loss.backward()
        optim.step()


        # aggregate values for metric calculation
        epoch_train_loss.append(loss.item())


        preds = {k:torch.argmax(v,dim=1) for k,v in out.items()}    
        epoch_metrics.append(get_multilevel_metrics(preds, batch, cfg))
        del batch, out
        torch.cuda.empty_cache()
        



    epoch_metrics = combine_metrics_list(epoch_metrics,cfg)
    print_metrics(epoch_metrics, cfg)
    print(f"L_train:{np.mean(epoch_train_loss):.4f} ")
    
    # validate  
    epoch_metrics = []
    epoch_val_loss = []
    model.eval()
    c = 0
    with torch.no_grad():
        for i,batch in enumerate(tqdm(val_dl, desc=f"{'Validation':<15}", leave=True)):
            # put batch on device
            for k, v in batch.items():
                batch[k] = v.to(device=cfg.general.device)

            # forward pass
            out = model(batch)
            # agregate loss on all levels
            loss = torch.tensor(0.).to(device=cfg.general.device)
            for k in out.keys():
                output = out[k]
                gt = batch[k] # type:ignore
                loss += criterion(output, gt)

            c+= 1
            if c > 100:
                raise Exception
        # aggregate values for metric calculation
        epoch_val_loss.append(loss.item())
        preds = {k:torch.argmax(v,dim=1) for k,v in out.items()}    
        epoch_metrics.append(get_multilevel_metrics(preds, batch, cfg))
        del batch, out
        torch.cuda.empty_cache()
            
    epoch_metrics = combine_metrics_list(epoch_metrics,cfg)
    print_metrics(epoch_metrics, cfg)


    print(f"L_val:{np.mean(epoch_val_loss):.4f}")
    print('\n')

    

Epoch-000
Sampling training dataset. Done. Took 13.48.


Training       : 100%|██████████| 885/885 [03:41<00:00,  4.00it/s]


                oA      mIoU        mP        mR       mF1     mcAcc
labels_3  0.794809  0.304755  0.387438  0.365058  0.361101  0.365058
L_train:0.6477 


Validation     :  12%|█▏        | 100/843 [00:04<00:36, 20.09it/s]


Exception: 

In [11]:
preds = {k:torch.argmax(v,dim=1) for k,v in out.items()}    

In [85]:
batch = {k:v.to(device='cuda') for k,v in batch.items()}
preds = {k:v.to(device='cuda') for k,v in preds.items()}

In [86]:
for el in preds.items():
    print(el[1].device)

cuda:0


In [92]:
get_multilevel_metrics(preds, batch, cfg)['labels_3']['classwise']['iou']

array([0.        , 0.20454545, 0.        , 0.        , 0.2682927 ,
       0.        , 0.        , 0.6538461 , 0.        , 0.        ,
       0.        , 0.97534835], dtype=float32)

In [42]:
(preds['labels_3'].detach().cpu().numpy() == batch['labels_3'].detach().cpu().numpy()).mean()

0.935546875

In [22]:
from torchmetrics.segmentation import MeanIoU

In [62]:
batch['labels_3'] = batch['labels_3'].to(dtype=torch.int64,device='cpu')

In [63]:
preds['labels_3']=preds['labels_3'].to(dtype=torch.int64,device='cpu')

In [50]:
preds['labels_3'].shape

torch.Size([1024])

In [101]:
cfg.data.label_schema.keys()

dict_keys(['labels_0', 'labels_1', 'labels_2', 'labels_3'])

In [105]:
train_ds.label_names

['labels_3']

In [100]:
miou = MeanIoU(num_classes=len(cfg.data.label_schema['labels_3']),input_format='index', per_class=True)
miou(preds['labels_3'][:,None], batch['labels_3'][:,None])

RuntimeError: Encountered different devices in metric calculation (see stacktrace for details). This could be due to the metric class not being on the same device as input. Instead of `metric=MeanIoU(...)` try to do `metric=MeanIoU(...).to(device)` where device corresponds to the device of the input.

In [99]:
from sklearn.metrics import jaccard_score
jaccard_score(
    batch['labels_3'].detach().cpu().numpy(),
    preds['labels_3'].detach().cpu().numpy(),average='micro')

0.8788990825688073

In [10]:
out

{'labels_3': tensor([[-1.6561, -6.4180, -3.8302,  ..., -4.6453,  1.2041,  3.8922],
         [-1.9462, -8.3578, -4.9595,  ..., -5.9825,  1.9146,  4.9588],
         [-1.9676, -8.5061, -5.0889,  ..., -5.9469,  1.9776,  4.9908],
         ...,
         [-0.7784,  3.9277, -4.6603,  ..., -2.9421, -5.2739,  0.4067],
         [-1.7697, -7.3003, -4.4040,  ..., -5.2239,  1.4652,  4.3442],
         [-1.8289, -7.4214, -4.5581,  ..., -5.4036,  1.4179,  4.4742]],
        device='cuda:0')}

In [11]:
batch['features_neibors'].shape

torch.Size([1024, 128, 16])

In [ ]:
# Epoch-049
# Sampling training dataset. Done. Took 22.75.
# Training       : 100%|██████████| 885/885 [07:28<00:00,  1.98it/s]
#                 oA      mIoU        mP        mR       mF1     mcAcc
# labels_0  0.965222  0.685658  0.766195  0.723284  0.732392  0.723284
# L_train:0.1072 
# Validation     : 100%|██████████| 843/843 [00:21<00:00, 39.72it/s]
#                 oA      mIoU        mP        mR       mF1     mcAcc
# labels_0  0.924567  0.308189  0.333333  0.308189  0.320268  0.308189
# L_val:0.2315

In [59]:
cfg.data.label_schema['labels_3']

{'_unspecified': 0, 'asphalt': 1, 'brick': 2, 'cable': 3, 'concrete': 4, 'marking': 5, 'mesh': 6, 'metal': 7, 'naturalStone': 8, 'poster': 9, 'treeTrunk': 10, 'vegetation': 11}